# Scrapping from the web

### Importing Libraries

In [50]:
from bs4 import BeautifulSoup
import requests

import pandas as pd
import numpy as np

### Get the destination page

In [51]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
print(page)

<Response [200]>


### Parsing the web page and getting table

In [49]:
soup = BeautifulSoup(page.content, 'html.parser')

In [53]:
#Getting the table
table = soup.find('table', {"class" : "wikitable sortable"})
print(table.prettify())

<table class="wikitable sortable">
 <tbody>
  <tr>
   <th>
    Postcode
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighbourhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Parkwoods" title="Parkwoods">
     Parkwoods
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Victoria_Village" title="Victoria Village">
     Victoria Village
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
     Downtown Toronto
    </a>
   </td>
   <td>
    <a href="

In [127]:
#Getting Values
Toronto = []

#Loop through tables to get the values from the 3 columns
for row in table.find_all('tr'):
    col = row.find_all('td')
    if len(col) == 3:
        Toronto.append((col[0].text.strip(), col[1].text.strip(), col[2].text.strip()))


### Creating arrays and data frame

In [128]:
#Converting the list to an array
Toronto = np.asarray(Toronto)

In [129]:
#Converting the array to a data frame
Toronto = pd.DataFrame(Toronto)
print(Toronto.shape)
Toronto.head()

(289, 3)


,0,1,2
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [130]:
#Naming the columns
names = ['PostalCode', 'Borough', 'Neighborhood']
Toronto.columns = names
Toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Handling the Not assigned values

In [131]:
#Dropping the values with Not assigned Borough
Toronto = Toronto[Toronto.Borough != 'Not assigned']
print(Toronto.shape)
print(Toronto.head())

(212, 3)
  PostalCode           Borough      Neighborhood
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront
5        M5A  Downtown Toronto       Regent Park
6        M6A        North York  Lawrence Heights


In [132]:
#Reseting the index
Toronto = Toronto.reset_index(drop = True)
print(Toronto.shape)
print(Toronto.head())

(212, 3)
  PostalCode           Borough      Neighborhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights


In [133]:
#Making the Not Assigned Neighborhood, same as the value of Borough
Toronto.loc[Toronto.Neighborhood == 'Not assigned','Neighborhood'] = Toronto.loc[Toronto.Neighborhood == 'Not assigned','Borough']

In [134]:
#Checking for the not assigned row
Toronto.loc[6]

PostalCode               M7A
Borough         Queen's Park
Neighborhood    Queen's Park
Name: 6, dtype: object

In [137]:
Toronto = Toronto.groupby('PostalCode').agg({'Borough':'first','Neighborhood':', '.join}).reset_index()

### Final look of the Data Frame

In [138]:
Toronto.head(12)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [141]:
Toronto.shape

(103, 3)